# Recipes scraping

## Spiders

In [1]:
import requests
from fake_useragent import UserAgent
from lxml import html, etree
import unicodedata
import locale
import time

locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [116]:
class BaseScrapper:

    def __init__(self, with_item_page: bool = True, user_agent: str = UserAgent().random, crawl_delay: int = 0):
        self._with_item_page = with_item_page
        self._crawl_delay = crawl_delay
        self._last_item_urls: set = set()
        self._data_to_return: list = []
        self._session = requests.Session()
        self._session.headers.update({'User-Agent': user_agent, })

    def __iter__(self) -> iter:
        return self

    def __next__(self):
        if len(self._data_to_return) == 0:
            data = self._crawl_data()
            item_urls = set([item['url'] for item in data])
            if self._last_item_urls != item_urls:
                self._last_item_urls = item_urls
                self._data_to_return = data
                return self.__next__()
            else:
                raise StopIteration
        else:
            return self._data_to_return.pop()

    def _crawl_data(self) -> list:
        url = self._next_url()
        page = self._get_page(url)
        data = self._extract_data(url, page)
        if self._with_item_page:
            return self._crawl_page_data(data)
        else:
            return data

    def _crawl_page_data(self, data) -> list:
        new_data = []
        for item in data:
            page_url = self._get_item_page(item)
            page = self._get_page(page_url)
            try:
                parsed_page = self._parse_item_page(page)
            except Exception:
                parsed_page = dict()
            merged = self._merge(item, parsed_page)
            new_data.append(merged)
        return new_data

    def _get_page(self, url) -> html.HtmlElement:
        time.sleep(self._crawl_delay)
        content = self._session.get(url).text
        #         if (self._session.status_code != requests.codes.ok):
        #             raise Exception('Response code', self._session.status_code, 'for url', url)
        parsed = html.fromstring(content)
        return parsed

    def _extract_data(self, url: str, parsed: html.HtmlElement) -> list:
        data = []
        items = self._get_items(parsed)
        for item in items:
            try:
                parsed = self._parse_item(item)
                data.append(parsed)
            except Exception:
                print('item fail:', url)
        return data

    def _next_url(self) -> str:
        raise NotImplementedError

    def _get_items(self, page: html.HtmlElement) -> list:
        raise NotImplementedError

    def _parse_item(self, item: html.HtmlElement) -> dict:
        raise NotImplementedError

    def _get_item_page(self, item) -> str:
        raise NotImplementedError

    def _parse_item_page(self, page: html.HtmlElement) -> dict:
        raise NotImplementedError

    def _merge(self, item: dict, page: dict) -> dict:
        raise NotImplementedError

In [136]:
class RussianFoodScrapper(BaseScrapper):

    def __init__(self, with_item_page: bool = True, user_agent: str = UserAgent().random, crawl_delay: int = 1):
        super().__init__(with_item_page=with_item_page, user_agent=user_agent, crawl_delay=crawl_delay)

    def _next_url(self) -> str:
        host = 'https://www.russianfood.com'
        start_page = 127
        end_page = 220
        if not hasattr(self, '_cur_page'):
            self._cur_page = start_page
        elif self._cur_page <= end_page:
            self._cur_page += 1
        return f'{host}/?page={self._cur_page}'

    def _get_items(self, page: html.HtmlElement) -> list:
        return page.xpath('//div[@class="annonce annonce_orange"]')

    def _parse_item(self, item: html.HtmlElement) -> dict:
        return {
            'raw_head': etree.tostring(item),
            'url': 'https://www.russianfood.com' + item.xpath('.//table[@class="top"]//td[@class="tc22"]//td[@class="toptext"]/p/a[@class="title"]/@href')[0],
        }

    def _get_item_page(self, item) -> str:
        return item['url']

    def _parse_item_page(self, page: html.HtmlElement) -> dict:
        return {
            'raw_body': etree.tostring(page.xpath('.//table[@class="recipe_new "]')[0]),
            'name': self._text(page.xpath('//table[@class="recipe_new "]//h1[@class="title "]/text()')[0]),
            'ingr': self._parse_ingr(page.xpath('//table[@class="recipe_new "]//table[@class="ingr_block"]//table[@class="ingr"]//tr[contains(@class, "ingr")]//span/text()')),
            'ingr_ext': self._parse_ingr_ext(page.xpath('//table[@class="recipe_new "]//table[@class="ingr_block"]//table[@class="ingr"]//tr[contains(@class, "ingr")]//span/text()')),
            'recipe': self._text(' '.join(page.xpath('//table[@class="recipe_new "]//table[@class="step_images"]//tr//p/text()'))).replace('\r\n', ' ').replace('\n', ' '),
            'complexity': len(self._parse_ingr_ext(page.xpath('//table[@class="recipe_new "]//table[@class="ingr_block"]//table[@class="ingr"]//tr[contains(@class, "ingr")]//span/text()'))),
        }

    def _merge(self, item: dict, page: dict) -> dict:
        return {**item, **page}

    def _text(self, s) -> str:
        return unicodedata.normalize('NFKD', s)

    def _parse_ingr(self, items):
        products = [self._text(item).split('-')[0].split() for item in items]
        return [item.lower() for sublist in products for item in sublist]

    def _parse_ingr_ext(self, items):
        return [self._text(item) for item in items]

## Persisting

Create mongodb docker container

In [96]:
!docker run -d -p 27017:27017 --name mongodb mongo

51562b656f1aee10947680690bda04b552f622e5c4119a4959eb20713b76f935


Start mongo db

In [ ]:
!docker start mongodb

Run spiders and fill database

In [3]:
from pymongo import MongoClient

In [137]:
mongo = MongoClient('localhost', 27017)
db = mongo.recipes

In [139]:
def fill_via_scrapper(recipes, collections, update_only=False):
    k = 0
    for recipe in recipes:
        k += 1
        matched = [collection.update_one({'_id': recipe['url']}, {'$set': recipe}, upsert=True).matched_count for collection in collections]
        if update_only and sum(matched) > 0:
            break
        if k % 10 == 0:
            print(k)

In [140]:
fill_via_scrapper(RussianFoodScrapper(), [db.dishes])

Get dump from mongodb container

In [126]:
!docker exec -it mongodb mongodump --out=/backup/ --db=recipes
!docker exec -it mongodb tar czf dump.mongo.tgz /backup
!docker cp mongodb:/dump.mongo.tgz dump.mongo.tgz
!docker exec -it mongodb rm -rf /backup /dump.mongo.tgz

2020-02-29T13:42:56.331+0000	writing recipes.dishes to 
2020-02-29T13:42:57.165+0000	done dumping recipes.dishes (1277 documents)
tar: Removing leading `/' from member names


Put dump into mongodb container

In [ ]:
!docker cp dump.mongo.tgz mongodb:/dump.mongo.tgz
!docker exec -it mongodb tar xzf dump.mongo.tgz
!docker exec -it mongodb mongorestore /backup
!docker exec -it mongodb rm -rf /backup /dump.mongo.tgz

Shutdown mongo db

In [ ]:
!docker stop mongodb

## Data validation

In [ ]:
from pprint import pprint

In [2]:
def get_data(ingr, collection, limit):
    for recipe in collection.find({'ingr': ingr}, {'name': 1, 'ingr': 1, 'url': 1}):
        print(recipe['name'])
        print('url:', recipe['url'])
        pprint(recipe['ingr'])
        print()
        limit -= 1
        if limit <= 0:
            break

In [ ]:
get_data('фарш', food_db.recipes, 3)

## Test data

### fill data

In [6]:
recipe1 = {
    '_id': 'test1', 
    'tags': ['вегетерианский', 'китайский'],
    'ingr': ['яйцо', 'яблоко'],
    'recipe': 'Выпейте вырое яйцо, закусывая яблоком',
    'name': 'Суровая жизнь'
}

recipe2 = {
    '_id': 'test2', 
    'ingr': ['яйцо', 'хлеб', 'масло'],
    'recipe': 'Пожарьте яйцо и съешьте с хлебом',
    'name': 'Глазунья'
}

for recipe in [recipe1, recipe2]:
    db.test.update_one({'_id': recipe['_id']}, {'$set': recipe}, upsert=True)

### read data

In [27]:
[x for x in db.test.find({"ingr": {"$all": ["яйцо", "яблоко"]}}, {'_id': 0, 'raw': 0})]

[{'ingr': ['яйцо', 'яблоко'],
  'name': 'Суровая жизнь',
  'recipe': 'Выпейте вырое яйцо, закусывая яблоком',
  'tags': ['вегетерианский', 'китайский']}]

In [5]:
def split_text(text):
    s = text.split('.')
    k = 0
    i = 0
    a = ['']
    for w in s:
        if k + len(w) + 2 > 500:
            k = 0
            i += 1
            a.append('')
        else:
            k += len(w) + 2
        a[i] += w + '. '
    return a

text = 'Подготовить составляющие слоеного салата с килькой в томате.   Картофель и морковь отварить в подсоленной воде до готовности (20-25 минут), достать из отвара и остудить. Остывшие отварные овощи очистить.  Яйцо залить холодной водой, отварить вкрутую (8 минут) и остудить в холодной воде. Остывшее яйцо очистить.  Репчатый лук очистить. Также для маринования лука понадобится холодная вода, уксус, сахар и соль. Лук мелко нарезать, добавить к нему 50 мл холодной воды, уксус, соль и сахар. Перемешать и оставить на 10 минут. Морковь натереть на крупной тёрке. Яйцо натереть на крупной тёрке. Огурцы тоже натереть на крупной тёрке. Отжать сок. Картофель натереть на крупной тёрке. Кильку достать из банки и выложить на тарелку. Большую часть соуса оставьте в банке, он не понадобится.  Размять кильку вилкой. На плоскую тарелку установить кулинарное кольцо (диаметр - 8 см).  Первым слоем выложить картофель и утрамбовать ложкой или прессом для салатов. Смазать майонезом. Лук откинуть на сито, чтобы стёк весь маринад, и выложить следующим слоем. Поверх распределить кильку. Следом выложить морковь, утрамбовать и смазать майонезом. Если позволяет высота кольца, повторить слой картофеля.  На морковь выложить огурцы, утрамбовать и тоже смазать майонезом. Последним слоем выложить яйца. Оставить салат в холодильнике на полчаса. Снять кулинарное кольцо. Украсить слоёный салат с килькой в томате и солёными огурцами зеленью петрушки и подать.  Приятного аппетита!'
[print(s, '\n') for s in split_text(text)]

Подготовить составляющие слоеного салата с килькой в томате.    Картофель и морковь отварить в подсоленной воде до готовности (20-25 минут), достать из отвара и остудить.  Остывшие отварные овощи очистить.   Яйцо залить холодной водой, отварить вкрутую (8 минут) и остудить в холодной воде.  Остывшее яйцо очистить.   Репчатый лук очистить.  Также для маринования лука понадобится холодная вода, уксус, сахар и соль.  Лук мелко нарезать, добавить к нему 50 мл холодной воды, уксус, соль и сахар.  

 Перемешать и оставить на 10 минут.  Морковь натереть на крупной тёрке.  Яйцо натереть на крупной тёрке.  Огурцы тоже натереть на крупной тёрке.  Отжать сок.  Картофель натереть на крупной тёрке.  Кильку достать из банки и выложить на тарелку.  Большую часть соуса оставьте в банке, он не понадобится.   Размять кильку вилкой.  На плоскую тарелку установить кулинарное кольцо (диаметр - 8 см).   Первым слоем выложить картофель и утрамбовать ложкой или прессом для салатов.  Смазать майонезом.  

 Лук

[None, None, None]

In [7]:
[x for x in db.test.find({})]

[{'_id': 'test1',
  'ingr': ['яйцо', 'яблоко'],
  'name': 'Суровая жизнь',
  'recipe': 'Выпейте вырое яйцо, закусывая яблоком',
  'tags': ['вегетерианский', 'китайский'],
  'complexity': 2,
  'recipe_size': 6},
 {'_id': 'test2',
  'ingr': ['яйцо', 'хлеб', 'масло'],
  'name': 'Глазунья',
  'recipe': 'Пожарьте яйцо и съешьте с хлебом',
  'complexity': 3,
  'recipe_size': 6}]

In [9]:
[x for x in db.test.find({'$query': {"ingr": {"$in": ["масло", "яблоко"]}}, "$orderby": { 'complexity' : 1 }}, {'_id': 0, 'raw': 0}).limit(1)]

[{'ingr': ['яйцо', 'яблоко'],
  'name': 'Суровая жизнь',
  'recipe': 'Выпейте вырое яйцо, закусывая яблоком',
  'tags': ['вегетерианский', 'китайский'],
  'complexity': 2,
  'recipe_size': 6}]